## Load dataset

In [1]:
import glob
import pandas as pd

import numpy as np

def highlight_max(data):
    percent = data.str.contains('%').any()

    data = data.str.split(expand=True)
    percent = data[0].str.contains('%').any()

    data[0] = data[0].replace('\\\%','', regex=True).replace('\$','', regex=True).astype(float)
    data[0] = np.where(data[0] == data[0].max(), '\mathbf{'+ str(data[0].max()) + '}', data[0])
    if percent:
        return '$' + data[0] + '\% ' + data[1] + ' ' + data[2]
    else:
        return '$' + data[0] + ' ' + data[1] + ' ' + data[2]

    return data

dataset_path = 'full'
#dataset_path = 'amauri'
#dataset_path = 'full_with_tfidf'
#dataset_path = 'small'

files = glob.glob(f'../evaluate_results/{dataset_path}/outer/*.csv', recursive=True)

print(len(files), 'files')
files[0:3]

40 files


["../evaluate_results/full/outer/LogisticRegression-split_x_y_split_with_one_hot_encoding-{'multi_class': ['auto'], 'solver': ['liblinear']}-cross_entropy-(1 of 5).csv",
 "../evaluate_results/full/outer/LogisticRegression-split_x_y_with_bag_of_words-{'multi_class': ['auto'], 'solver': ['liblinear']}-cross_entropy-(2 of 5).csv",
 "../evaluate_results/full/outer/MLPClassifier-split_x_y_split_with_one_hot_encoding-{'hidden_layer_sizes': [20, 40, 80, 100], 'max_iter': [800]}-accuracy-(3 of 5).csv"]

In [2]:
# Read all data
data = pd.concat([pd.read_csv(file, index_col=[0]) for file in files])
data.head(2)

,best_params,column,i_outer,metric,model,params,refit,split_method,value
0,"{'multi_class': 'auto', 'solver': 'liblinear'}",0,0,accuracy,LogisticRegression,"{'multi_class': ['auto'], 'solver': ['liblinea...",cross_entropy,split_x_y_split_with_one_hot_encoding,0.475751
1,"{'multi_class': 'auto', 'solver': 'liblinear'}",0,0,hit@5,LogisticRegression,"{'multi_class': ['auto'], 'solver': ['liblinea...",cross_entropy,split_x_y_split_with_one_hot_encoding,0.676674


## Results

In [3]:
result = data.groupby(['model', 'split_method', 'metric'])['value'].agg(["mean", "std"])
result.head(3)

mean  \
model                split_method                          metric               
KNeighborsClassifier split_x_y_split_with_one_hot_encoding accuracy  0.356083   
                                                           hit@5     0.525216   
                                                           map@5     0.303942   

                                                                          std  
model                split_method                          metric              
KNeighborsClassifier split_x_y_split_with_one_hot_encoding accuracy  0.131148  
                                                           hit@5     0.128174  
                                                           map@5     0.155788

## Result as pivot table

(Beauty table bellow)

In [4]:
pivot_table = result.pivot_table(index=['model', 'split_method'], columns='metric', values=['mean', 'std'])
pivot_table

mean  \
metric                                                      accuracy   
model                split_method                                      
KNeighborsClassifier split_x_y_split_with_one_hot_encoding  0.356083   
                     split_x_y_with_bag_of_words            0.342662   
LogisticRegression   split_x_y_split_with_one_hot_encoding  0.389859   
                     split_x_y_with_bag_of_words            0.342660   
MLPClassifier        split_x_y_split_with_one_hot_encoding  0.347435   
                     split_x_y_with_bag_of_words            0.352223   
SVC                  split_x_y_split_with_one_hot_encoding  0.394791   
                     split_x_y_with_bag_of_words            0.380994   

                                                                          \
metric                                                     cross_entropy   
model                split_method                                          
KNeighborsClassifier split_x_y_split_with_one_hot_encoding           NaN   
                     split_x_y_with_bag_of_words                     NaN   
LogisticRegression   split_x_y_split_with_one_hot_encoding      2.778379   
                     split_x_y_with_bag_of_words                3.131092   
MLPClassifier        split_x_y_split_with_one_hot_encoding           NaN   
                     split_x_y_with_bag_of_words                     NaN   
SVC                  split_x_y_split_with_one_hot_encoding           NaN   
                     split_x_y_with_bag_of_words                     NaN   

                                                                      \
metric                                                         hit@5   
model                split_method                                      
KNeighborsClassifier split_x_y_split_with_one_hot_encoding  0.525216   
                     split_x_y_with_bag_of_words            0.520976   
LogisticRegression   split_x_y_split_with_one_hot_encoding  0.593544   
                     split_x_y_with_bag_of_words            0.515190   
MLPClassifier        split_x_y_split_with_one_hot_encoding  0.552361   
                     split_x_y_with_bag_of_words            0.560688   
SVC                  split_x_y_split_with_one_hot_encoding  0.585606   
                     split_x_y_with_bag_of_words            0.582834   

                                                                      \
metric                                                         map@5   
model                split_method                                      
KNeighborsClassifier split_x_y_split_with_one_hot_encoding  0.303942   
                     split_x_y_with_bag_of_words            0.300250   
LogisticRegression   split_x_y_split_with_one_hot_encoding  0.325379   
                     split_x_y_with_bag_of_words            0.317795   
MLPClassifier        split_x_y_split_with_one_hot_encoding  0.307099   
                     split_x_y_with_bag_of_words            0.319760   
SVC                  split_x_y_split_with_one_hot_encoding  0.334276   
                     split_x_y_with_bag_of_words            0.333675   

                                                                      \
metric                                                          mdcg   
model                split_method                                      
KNeighborsClassifier split_x_y_split_with_one_hot_encoding  0.544766   
                     split_x_y_with_bag_of_words            0.539627   
LogisticRegression   split_x_y_split_with_one_hot_encoding  0.591116   
                     split_x_y_with_bag_of_words            0.544051   
MLPClassifier        split_x_y_split_with_one_hot_encoding  0.556736   
                     split_x_y_with_bag_of_words            0.563953   
SVC                  split_x_y_split_with_one_hot_encoding  0.584850   
                     split_x_y_with_bag_of_words            0.582085   

                                                       

### 'Cute' Pivot table

In [5]:
data = pd.read_csv('../rbm_results-early-10000.csv', index_col=0)

data['metric'].replace('Accuracy', 'accuracy', inplace=True)
data['metric'].replace('Hit@5', 'hit@5', inplace=True)
data['metric'].replace('MAP@5', 'map@5', inplace=True)
data['metric'].replace('MDCG', 'mdcg', inplace=True)


data['total_missing'] = data['missing'].str.split(',').str.len() + 1
data['total_missing'] = np.where(data['missing'] == 'set()', 1, data['total_missing'])
rbm_pivot = data.groupby(['metric', 'total_missing'])['value'].agg(["mean", "std"]) \
    .pivot_table(index=['total_missing'], columns='metric', values=['mean', 'std'])
rbm_pivot.iloc[0]
rbm_hide_1 = rbm_pivot.iloc[0].to_frame().T
rbm_hide_1.index = [('RBM', 'split_x_y_split_with_one_hot_encoding')]
rbm_hide_1

mean                      \
metric                                        accuracy     hit@5     map@5   
(RBM, split_x_y_split_with_one_hot_encoding)  0.308956  0.504236  0.310148   

                                                             std            \
metric                                            mdcg  accuracy     hit@5   
(RBM, split_x_y_split_with_one_hot_encoding)  0.526222  0.138047  0.134807   

                                                                 
metric                                           map@5     mdcg  
(RBM, split_x_y_split_with_one_hot_encoding)  0.153247  0.10864

In [6]:
def format_percentage(x):
    return '{:2.2%}'.format(x).replace('%', '\%')

def format_std(x):
    return '{:.4f}'.format(x)

pivot_table2 = pd.concat([pivot_table, rbm_hide_1])

new_pivot_table = pd.DataFrame({
    'Accuracy (std)': '$' + pivot_table2['mean']['accuracy'].map(format_percentage) + ' \pm ' + pivot_table2['std']['accuracy'].map(format_percentage) + "$",
    'Hit@5 (std)':    '$' + pivot_table2['mean']['hit@5'].map(format_percentage)    + ' \pm ' + pivot_table2['std']['hit@5'].map(format_percentage) + "$",
    'MDCG (std)':     '$' + pivot_table2['mean']['mdcg'].map(format_std)            + ' \pm ' + pivot_table2['std']['mdcg'].map(format_std) + "$",
    'MAP@5 (std)':    '$' + pivot_table2['mean']['map@5'].map(format_percentage)    + ' \pm ' + pivot_table2['std']['map@5'].map(format_percentage) + "$",
})

new_pivot_table.index.set_names(['Model','Embedding'],inplace=True)
new_pivot_table.rename(index={
    'split_x_y': 'No embedding',
    'split_x_y_split_with_one_hot_encoding': 'One-hot concatenated',
    'split_x_y_with_bag_of_words': 'Bag-of-words',
    'KNeighborsClassifier': '$k$-NN',
    'LogisticRegression': 'Logistic Regression',
    'MLPClassifier': 'MLP',
}, inplace=True)

new_pivot_table.apply(highlight_max).to_latex("table.tex", escape=False)
new_pivot_table.apply(highlight_max)


Accuracy (std)  \
Model               Embedding                                              
$k$-NN              One-hot concatenated           $35.61\% \pm 13.11\%$   
                    Bag-of-words                   $34.27\% \pm 13.58\%$   
Logistic Regression One-hot concatenated           $38.99\% \pm 12.81\%$   
                    Bag-of-words                   $34.27\% \pm 13.71\%$   
MLP                 One-hot concatenated           $34.74\% \pm 12.72\%$   
                    Bag-of-words                   $35.22\% \pm 13.06\%$   
SVC                 One-hot concatenated  $\mathbf{39.48}\% \pm 12.22\%$   
                    Bag-of-words                    $38.1\% \pm 13.08\%$   
RBM                 One-hot concatenated            $30.9\% \pm 13.80\%$   

                                                             Hit@5 (std)  \
Model               Embedding                                              
$k$-NN              One-hot concatenated           $52.52\% \pm 12.82\%$   
                    Bag-of-words                    $52.1\% \pm 13.15\%$   
Logistic Regression One-hot concatenated  $\mathbf{59.35}\% \pm 11.38\%$   
                    Bag-of-words                   $51.52\% \pm 14.05\%$   
MLP                 One-hot concatenated           $55.24\% \pm 11.12\%$   
                    Bag-of-words                   $56.07\% \pm 11.75\%$   
SVC                 One-hot concatenated           $58.56\% \pm 12.03\%$   
                    Bag-of-words                   $58.28\% \pm 12.07\%$   
RBM                 One-hot concatenated           $50.42\% \pm 13.48\%$   

                                                            MDCG (std)  \
Model               Embedding                                            
$k$-NN              One-hot concatenated           $0.5448 \pm 0.1037$   
                    Bag-of-words                   $0.5396 \pm 0.1076$   
Logistic Regression One-hot concatenated  $\mathbf{0.5911} \pm 0.0970$   
                    Bag-of-words                   $0.5441 \pm 0.1094$   
MLP                 One-hot concatenated           $0.5567 \pm 0.0965$   
                    Bag-of-words                    $0.564 \pm 0.0996$   
SVC                 One-hot concatenated           $0.5849 \pm 0.0997$   
                    Bag-of-words                   $0.5821 \pm 0.1016$   
RBM                 One-hot concatenated           $0.5262 \pm 0.1086$   

                                                             MAP@5 (std)  
Model               Embedding                                             
$k$-NN              One-hot concatenated           $30.39\% \pm 15.58\%$  
                    Bag-of-words                   $30.03\% \pm 15.65\%$  
Logistic Regression One-hot concatenated           $32.54\% \pm 15.36\%$  
                    Bag-of-words                   $31.78\% \pm 16.07\%$  
MLP                 One-hot concatenated           $30.71\% \pm 15.17\%$  
                    Bag-of-words                   $31.98\% \pm 14.99\%$  
SVC                 One-hot concatenated  $\mathbf{33.43}\% \pm 15.65\%$  
                    Bag-of-words                   $33.37\% \pm 15.57\%$  
RBM                 One-hot concatenated           $31.01\% \pm 15.32\%$

###### Load dataset (hide two columns)

### Two hidden columns

In [7]:
files = glob.glob(f'../evaluate_results/hide_two/outer/*.csv', recursive=True)

# Read all data
data_hide_two = pd.concat([pd.read_csv(file, index_col=[0]) for file in files])

result = data_hide_two.groupby(['model', 'split_method', 'metric', 'column'])['value'].agg(["mean", "std"])
pivot_table = result.pivot_table(index=['model', 'split_method', 'column'], columns='metric', values=['mean', 'std'])


new_pivot_table = pd.DataFrame({
    'Accuracy': '$' + pivot_table['mean']['accuracy'].map(format_percentage) + ' \pm ' + pivot_table['std']['accuracy'].map(format_percentage) + "$",
    'Hit@5':    '$' + pivot_table['mean']['hit@5'].map(format_percentage) + ' \pm ' + pivot_table['std']['hit@5'].map(format_percentage) + "$",
    'MDCG':     '$' + pivot_table['mean']['mdcg'].map(format_std) + ' \pm ' + pivot_table['std']['mdcg'].map(format_std) + "$",
    'MAP@5':    '$' + pivot_table['mean']['map@5'].map(format_percentage) + ' \pm ' + pivot_table['std']['map@5'].map(format_percentage) + "$",
})

new_pivot_table.index.set_names(['Model','Embedding', 'Column'],inplace=True)
new_pivot_table.rename(index={
    'split_x_y': 'No embedding',
    'split_x_y_split_with_one_hot_encoding': 'One-hot concatenated',
    'split_x_y_with_bag_of_words': 'Bag-of-words',
    'KNeighborsClassifier': '$k$-NN',
    'LogisticRegression': 'Logistic Regression',
    'MLPClassifier': 'MLP',
}, inplace=True)

print("Duas colunas são ocultadas. A indicada em 'Column' é o índice da coluna colocada como 'y'. A outra coluna ocultada segue a seguinte regra")
print("Obs: A 5º coluna é mostrada como '3' pq o índice começa como zero e pq eu tiro a 2º coluna para computar os dados")
print("  - if column = 1, then the other hidden column is 5: P(column_1 | columns \ {column_1, column_5})")
print("  - if column = 5, then the other hidden column is 1: P(column_5 | columns \ {column_1, column_5})")
new_pivot_table.apply(highlight_max)

Duas colunas são ocultadas. A indicada em 'Column' é o índice da coluna colocada como 'y'. A outra coluna ocultada segue a seguinte regra
Obs: A 5º coluna é mostrada como '3' pq o índice começa como zero e pq eu tiro a 2º coluna para computar os dados
  - if column = 1, then the other hidden column is 5: P(column_1 | columns \ {column_1, column_5})
  - if column = 5, then the other hidden column is 1: P(column_5 | columns \ {column_1, column_5})


Accuracy  \
Model               Embedding            Column                                 
$k$-NN              No embedding         1               $20.18\% \pm 1.35\%$   
                                         3               $44.05\% \pm 2.42\%$   
Logistic Regression One-hot concatenated 1               $24.06\% \pm 2.25\%$   
                                         3               $46.65\% \pm 1.67\%$   
MLP                 One-hot concatenated 1               $20.92\% \pm 1.79\%$   
                                         3               $42.71\% \pm 0.89\%$   
SVC                 One-hot concatenated 1               $25.13\% \pm 1.95\%$   
                                         3       $\mathbf{47.2}\% \pm 1.46\%$   

                                                                         Hit@5  \
Model               Embedding            Column                                  
$k$-NN              No embedding         1                $39.75\% \pm 2.07\%$   
                                         3                $58.91\% \pm 2.03\%$   
Logistic Regression One-hot concatenated 1                $47.39\% \pm 1.18\%$   
                                         3       $\mathbf{66.87}\% \pm 0.92\%$   
MLP                 One-hot concatenated 1                 $43.5\% \pm 1.85\%$   
                                         3                $62.29\% \pm 1.81\%$   
SVC                 One-hot concatenated 1                $46.83\% \pm 1.89\%$   
                                         3                $66.45\% \pm 1.12\%$   

                                                                         MDCG  \
Model               Embedding            Column                                 
$k$-NN              No embedding         1                $0.4291 \pm 0.0177$   
                                         3                $0.6037 \pm 0.0177$   
Logistic Regression One-hot concatenated 1                $0.4815 \pm 0.0166$   
                                         3       $\mathbf{0.6516} \pm 0.0104$   
MLP                 One-hot concatenated 1                $0.4508 \pm 0.0153$   
                                         3                $0.6182 \pm 0.0034$   
SVC                 One-hot concatenated 1                $0.4769 \pm 0.0135$   
                                         3                $0.6509 \pm 0.0095$   

                                                                         MAP@5  
Model               Embedding            Column                                 
$k$-NN              No embedding         1                $14.59\% \pm 0.54\%$  
                                         3                $42.51\% \pm 1.45\%$  
Logistic Regression One-hot concatenated 1                $16.05\% \pm 0.51\%$  
                                         3                 $44.3\% \pm 1.33\%$  
MLP                 One-hot concatenated 1                $15.47\% \pm 0.13\%$  
                                         3                $42.27\% \pm 1.33\%$  
SVC                 One-hot concatenated 1                 $16.4\% \pm 0.75\%$  
                                         3       $\mathbf{44.48}\% \pm 1.22\%$

### Compare with one hidden column

In [8]:
pivot_table = data[
        (data.split_method == 'split_x_y_split_with_one_hot_encoding') 
      & ((data.column == 1) | (data.column == 4))] \
    .groupby(['model', 'split_method', 'metric', 'column'])['value'] \
    .agg(["mean", "std"]) \
    .pivot_table(index=['model', 'split_method', 'column'], columns='metric', values=['mean', 'std'])



new_pivot_table = pd.DataFrame({
    'Accuracy': '$' + pivot_table['mean']['accuracy'].map(format_percentage) + ' \pm ' + pivot_table['std']['accuracy'].map(format_percentage) + "$",
    'Hit@5':    '$' + pivot_table['mean']['hit@5'].map(format_percentage) + ' \pm ' + pivot_table['std']['hit@5'].map(format_percentage) + "$",
    'MDCG':     '$' + pivot_table['mean']['mdcg'].map(format_std) + ' \pm ' + pivot_table['std']['mdcg'].map(format_std) + "$",
    'MAP@5':    '$' + pivot_table['mean']['map@5'].map(format_percentage) + ' \pm ' + pivot_table['std']['map@5'].map(format_percentage) + "$",
})

new_pivot_table.index.set_names(['Model','Embedding', 'Column'],inplace=True)
new_pivot_table.rename(index={
    'split_x_y': 'No embedding',
    'split_x_y_split_with_one_hot_encoding': 'One-hot concatenated',
    'split_x_y_with_bag_of_words': 'Bag-of-words',
    'KNeighborsClassifier': '$k$-NN',
    'LogisticRegression': 'Logistic Regression',
    'MLPClassifier': 'MLP',
}, inplace=True)

print("Somente a coluna indicada está oculta")
print("A 5º coluna é 4 pq não deletei a 2º coluna")
print("  - P(column_n | columns \ {column_n})")

new_pivot_table

AttributeError: 'DataFrame' object has no attribute 'split_method'

Ao comparar as duas tabelas, se conclui que 

* remover a coluna 4 tira pouca informação na previsão da coluna 1;
* remover a coluna 1 tira pouca informação na previsão da coluna 4.

# First two columns

In [ ]:
files = glob.glob(f'../evaluate_results/hide_two_hard_p1/outer/*.csv', recursive=True)
files2 = glob.glob(f'../evaluate_results/hide_two_hard/outer/*.csv', recursive=True)

# Read all data
data_hide_two = pd.concat([pd.read_csv(file, index_col=[0]) for file in files])
data_hide_two['column_y'] = 1
data_hide_two['column_ignored'] = 0
data_hide_two2 = pd.concat([pd.read_csv(file, index_col=[0]) for file in files2])
data_hide_two2['column_y'] = 0
data_hide_two2['column_ignored'] = 1

data_hide_two = pd.concat((data_hide_two, data_hide_two2))

result = data_hide_two.groupby(['model', 'split_method', 'metric', 'column_y', 'column_ignored'])['value'].agg(["mean", "std"])
pivot_table = result.pivot_table(index=['model', 'split_method', 'column_y', 'column_ignored'], columns='metric', values=['mean', 'std'])


new_pivot_table = pd.DataFrame({
    'Accuracy': '$' + pivot_table['mean']['accuracy'].map(format_percentage) + ' \pm ' + pivot_table['std']['accuracy'].map(format_percentage) + "$",
    'Hit@5':    '$' + pivot_table['mean']['hit@5'].map(format_percentage) + ' \pm ' + pivot_table['std']['hit@5'].map(format_percentage) + "$",
    'MDCG':     '$' + pivot_table['mean']['mdcg'].map(format_std) + ' \pm ' + pivot_table['std']['mdcg'].map(format_std) + "$",
    'MAP@5':    '$' + pivot_table['mean']['map@5'].map(format_percentage) + ' \pm ' + pivot_table['std']['map@5'].map(format_percentage) + "$",
})

new_pivot_table.index.set_names(['Model','Embedding', 'Column y', 'Column ignored'],inplace=True)
new_pivot_table.rename(index={
    'split_x_y': 'No embedding',
    'split_x_y_split_with_one_hot_encoding': 'One-hot concatenated',
    'split_x_y_with_bag_of_words': 'Bag-of-words',
    'KNeighborsClassifier': '$k$-NN',
    'LogisticRegression': 'Logistic Regression',
    'MLPClassifier': 'MLP',
}, inplace=True)

new_pivot_table

In [ ]:
pivot_table = data[
        (data.split_method == 'split_x_y_split_with_one_hot_encoding') 
      & ((data.column == 0) | (data.column == 1))] \
    .groupby(['model', 'split_method', 'metric', 'column'])['value'] \
    .agg(["mean", "std"]) \
    .pivot_table(index=['model', 'split_method', 'column'], columns='metric', values=['mean', 'std'])


new_pivot_table = pd.DataFrame({
    'Accuracy': '$' + pivot_table['mean']['accuracy'].map(format_percentage) + ' \pm ' + pivot_table['std']['accuracy'].map(format_percentage) + "$",
    'Hit@5':    '$' + pivot_table['mean']['hit@5'].map(format_percentage) + ' \pm ' + pivot_table['std']['hit@5'].map(format_percentage) + "$",
    'MDCG':     '$' + pivot_table['mean']['mdcg'].map(format_std) + ' \pm ' + pivot_table['std']['mdcg'].map(format_std) + "$",
    'MAP@5':    '$' + pivot_table['mean']['map@5'].map(format_percentage) + ' \pm ' + pivot_table['std']['map@5'].map(format_percentage) + "$",
})

new_pivot_table.index.set_names(['Model','Embedding', 'Column'],inplace=True)
new_pivot_table.rename(index={
    'split_x_y': 'No embedding',
    'split_x_y_split_with_one_hot_encoding': 'One-hot concatenated',
    'split_x_y_with_bag_of_words': 'Bag-of-words',
    'KNeighborsClassifier': '$k$-NN',
    'LogisticRegression': 'Logistic Regression',
    'MLPClassifier': 'MLP',
}, inplace=True)


new_pivot_table